## NFL Predictions Model

2024 Results: https://docs.google.com/presentation/d/1V8nD894wQmctfSbW4x1LnBecPViLVMpCaN5Pzcu_H24/edit#slide=id.g322fb2b0fd6_0_7

2024 regular season record: 186-86



In [1]:
!pip install nfl_data_py
!pip install pandas
!pip install os
!pip install matplotlib

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


Add google drive file mount

In [2]:
import nfl_data_py as nfl
import pandas as pd
import os
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [3]:

#upload nfl_elo data from 2020-2024 - Titled "nfl-win-totals_{insert year here}"
from google.colab import files
files.upload()


pbp_data = nfl.import_pbp_data([2019, 2020, 2021, 2022, 2023, 2024])
# pbp_2022 = nfl.import_pbp_data([2022])
win_totals = nfl.import_schedules([2019, 2020, 2021, 2022, 2023, 2024])

#VERY IMPORTANT: CHANGE YEAR
current_year = 2024

# pbp_clean_2021 = pbp_2021[(pbp_2021['play_type'] != 'qb_kneel') & (pbp_2021['play_type'] != 'no_play') & (pbp_2021['play_type'] != 'qb_spike')].dropna(subset=['play_type'])
# pbp_rp_2021 = pbp_2021[((pbp_2021['pass'] == 1) | (pbp_2021['rush']==1)) & (pbp_2021['play_type'] != 'no_play')]
# fourthq_2021 = pbp_clean_2021[(pbp_clean_2021['qtr'] == 4) & (abs(pbp_clean_2021['score_differential']) <= 8 )]
# clutch_2021 = fourthq_2021[(fourthq_2021['game_seconds_remaining'] <= 300)]
# drive_2021 = pbp_2021[pbp_2021['drive_play_id_started'] == pbp_2021['play_id']]
# fourth_quarter_drives_2021 = drive_2021[(drive_2021['qtr']==4)&(abs(pbp_clean_2021['score_differential']) <= 8 )]


Saving nfl-win-totals_2024.csv to nfl-win-totals_2024 (2).csv
Saving nfl-win-totals_2020.csv to nfl-win-totals_2020 (2).csv
Saving nfl-win-totals_2021.csv to nfl-win-totals_2021 (2).csv
Saving nfl-win-totals_2022.csv to nfl-win-totals_2022 (2).csv
Saving nfl-win-totals_2023.csv to nfl-win-totals_2023 (2).csv
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
Downcasting floats.


In [5]:

win_totals['away_win'] = win_totals.apply(lambda row : 0 if row['result']>0 else 1, axis=1)
win_totals['home_win'] = win_totals.apply(lambda row : 1 if row['result']>0 else 0, axis=1)
# home_wins = win_totals_2021.groupby('home_team')['home_win'].sum().reset_index()
# away_wins = win_totals_2021.groupby('away_team')['away_win'].sum().reset_index()
# wins_2021 = (home_wins['home_win'] + away_wins['away_win']).reset_index()
# wins_2021.drop(columns='index', inplace=True)
# wins_2021.rename(columns={0:'wins'},inplace=True)
# teams = home_wins[['home_team']].rename(columns={'home_team':'team'})
# wins_2021 = teams.join(wins_2021)
# wins_2021


In [6]:
###test
# win_totals_2021[win_totals_2021['season']==2022]
seasonal_data = nfl.import_seasonal_data([2019,2020,2021,2022, 2023])
depth_chart = nfl.import_depth_charts([2019,2020,2021,2022, 2023])
roster = nfl.import_seasonal_rosters([2019,2020,2021,2022,2023, 2024])


In [7]:
seasonal_data[seasonal_data['season']==2019]

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,00-0019596,2019,REG,373,613,4057.0,24,8.0,27.0,185.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.178921
4,00-0020531,2019,REG,281,378,2979.0,27,4.0,12.0,89.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.190652
6,00-0021206,2019,REG,3,5,24.0,0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004985
7,00-0022127,2019,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.086170,0.286362,0.107915,0.133333,0.117904,0.119691,0.120624,0.112999,0.886097,0.083894
9,00-0022787,2019,REG,50,67,580.0,3,1.0,2.0,19.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224,00-0035703,2019,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.010118,0.196293,0.073442,0.103448,0.072993,0.078313,0.088445,0.079443,0.578947,0.040574
2229,00-0035704,2019,REG,100,156,1020.0,7,3.0,5.0,26.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.184162
2233,00-0035710,2019,REG,284,459,3027.0,24,12.0,38.0,295.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.185911
2240,00-0035719,2019,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.222588,0.428641,0.206808,0.107143,0.171123,0.162791,0.156975,0.186875,1.754923,0.129103


In [8]:
###feature engineer the training data


fullteamdata_arr=[]
for i in range(2019,2024):
  pbp_temp = pbp_data[pbp_data['season']==i]
  # pbp_2022 = nfl.import_pbp_data([2022])
  win_totals_temp = win_totals[win_totals['season']==i]
  if i == 2019:
    win_totals_temp.replace(to_replace='OAK',value='LV',inplace=True)
  pbp_clean = pbp_temp[(pbp_temp['play_type'] != 'qb_kneel') & (pbp_temp['play_type'] != 'no_play') & (pbp_temp['play_type'] != 'qb_spike')].dropna(subset=['play_type'])
  pbp_rp = pbp_temp[((pbp_temp['pass'] == 1) | (pbp_temp['rush']==1)) & (pbp_temp['play_type'] != 'no_play')]
  fourthq = pbp_clean[(pbp_clean['qtr'] == 4) & (abs(pbp_clean['score_differential']) <= 8 )]
  clutch_data = fourthq[(fourthq['game_seconds_remaining'] <= 300)]
  drive_data = pbp_temp[pbp_temp['drive_play_id_started'] == pbp_temp['play_id']]
  fourth_quarter_drives = drive_data[(drive_data['qtr']==4)&(abs(pbp_clean['score_differential']) <= 8 )]
  #calculate win totals
  home_wins = win_totals_temp.groupby('home_team')['home_win'].sum().reset_index()
  away_wins = win_totals_temp.groupby('away_team')['away_win'].sum().reset_index()
  wins_data = (home_wins['home_win'] + away_wins['away_win']).reset_index()
  wins_data.drop(columns='index', inplace=True)
  wins_data.rename(columns={0:'wins'},inplace=True)
  teams = home_wins[['home_team']].rename(columns={'home_team':'team'})
  wins_data = teams.join(wins_data)

  #off/def epa
  off_epa_data = pbp_clean.groupby('posteam')[['epa','wpa']].mean().reset_index()
  off_epa_data.rename(columns={'epa':'off_epa', 'wpa':'off_wpa', 'posteam':'team'}, inplace=True)
  def_epa_data = pbp_clean.groupby('defteam')[['epa', 'wpa']].mean().reset_index()
  def_epa_data.rename(columns={'epa':'def_epa', 'wpa':'def_wpa', 'defteam':'team'},inplace=True)
  epa_data = pd.merge(off_epa_data,def_epa_data,on='team')
  fullteamdata = epa_data
  #run/pass epa/wpa
  off_epa_rp_data = pbp_rp.groupby('posteam')[['epa', 'wpa']].mean().reset_index()
  off_epa_rp_data.rename(columns={'epa':'offrp_epa', 'wpa':'offrp_wpa', 'posteam':'team'}, inplace=True)
  def_epa_rp_data = pbp_rp.groupby('defteam')[['epa','wpa']].mean().reset_index()
  def_epa_rp_data.rename(columns={'epa':'defrp_epa', 'wpa':'defrp_wpa', 'defteam':'team'},inplace=True)
  epa_rp_data = pd.merge(off_epa_rp_data,def_epa_rp_data,on='team')
  fullteamdata = fullteamdata.merge(epa_rp_data, how='inner', on='team')

  #four_min_offense
  four_min_data = pbp_clean[((pbp_clean['half_seconds_remaining']<=240.0) & (pbp_clean['game_half'] == 'Half1')) | \
                                    (clutch_data['game_seconds_remaining']<=240.0)]
  off_epa_fourmin_data = four_min_data.groupby('posteam')[['epa', 'wpa']].mean().reset_index().rename(columns={'epa':'off4min_epa', 'wpa':'off4min_wpa', 'posteam':'team'})
  def_epa_fourmin_data = four_min_data.groupby('defteam')[['epa', 'wpa']].mean().reset_index().rename(columns={'epa':'def4min_epa', 'wpa':'def4min_wpa', 'defteam':'team'})
  fourmin_epa_data = pd.merge(off_epa_fourmin_data, def_epa_fourmin_data, on='team')
  fullteamdata = fullteamdata.merge(fourmin_epa_data, on='team')
  #4q
  fourthq = pbp_clean[(pbp_clean['qtr'] == 4) & (abs(pbp_clean['score_differential']) <= 8 )]
  #4thq_oneposs
  fourthq_off_epa_data = fourthq.groupby("posteam")[['epa', 'wpa']].mean().reset_index().rename(columns={'epa':'off4q_epa', 'wpa':'off4q_wpa', 'posteam':'team'})
  fourthq_def_epa_data = fourthq.groupby("defteam")[['epa', 'wpa']].mean().reset_index().rename(columns={'epa':'def4q_epa', 'wpa':'def4q_wpa', 'defteam':'team'})
  fourthq_epa_data = pd.merge(fourthq_off_epa_data, fourthq_def_epa_data, on='team')
  fullteamdata = fullteamdata.merge(fourthq_epa_data, on='team')
  #clutch
  clutch_data = fourthq[fourthq['game_seconds_remaining'] <= 300]
  clutch_off_epa_data = clutch_data.groupby('posteam')[['epa', 'wpa']].mean().reset_index().rename(columns={'epa':'offclutch_epa', 'wpa':'offclutch_wpa', 'posteam':'team'})
  clutch_def_epa_data = clutch_data.groupby('defteam')[['epa', 'wpa']].mean().reset_index().rename(columns={'epa':'defclutch_epa', 'wpa':'defclutch_wpa', 'defteam':'team'})
  clutch_epa_data = pd.merge(clutch_off_epa_data, clutch_def_epa_data, on='team')
  fullteamdata = fullteamdata.merge(clutch_epa_data, on='team')
  #downs
  first_down_data = pbp_rp[pbp_rp['down'] == 1]
  second_down_data = pbp_rp[pbp_rp['down'] == 2]
  third_down_data = pbp_rp[(pbp_rp['down'] == 3)]
  fourth_down_data = pbp_rp[pbp_rp['down']==4 ]


  #ydstogo
  second_down_off_ydstogo_data = second_down_data.groupby('posteam')['ydstogo'].mean().reset_index().rename(columns = {'ydstogo':'ydstogo_2nd_off', 'posteam':'team'})
  second_down_def_ydstogo_data = second_down_data.groupby('defteam')['ydstogo'].mean().reset_index().rename(columns = {'ydstogo':'ydstogo_2nd_def', 'defteam':'team'})

  third_down_off_ydstogo_data = third_down_data.groupby('posteam')['ydstogo'].mean().reset_index().rename(columns={'ydstogo':'ydstogo_3rd_off', 'posteam':'team'})
  third_down_def_ydstogo_data = third_down_data.groupby('defteam')['ydstogo'].mean().reset_index().rename(columns = {'ydstogo':'ydstogo_3rd_def', 'defteam':'team'})

  ydstogo2nd_data = pd.merge(second_down_off_ydstogo_data, second_down_def_ydstogo_data, on='team')
  ydstogo3rd_data = pd.merge(third_down_off_ydstogo_data, third_down_def_ydstogo_data, on='team')
  fullteamdata = fullteamdata.merge(ydstogo2nd_data, on='team')
  fullteamdata = fullteamdata.merge(ydstogo3rd_data, on='team')

  #3rd/4th down
  third_down_off_data = third_down_data.groupby('posteam')['third_down_converted'].mean().reset_index().rename(columns = {'third_down_converted':'third_down_off', 'posteam':'team'})
  fourth_down_off_data = fourth_down_data.groupby('posteam')['fourth_down_converted'].mean().reset_index().rename(columns={'fourth_down_converted':'fourth_down_off', 'posteam':'team'})
  third_down_def_data = third_down_data.groupby('defteam')['third_down_converted'].mean().reset_index().rename(columns={'third_down_converted':'third_down_def', 'defteam':'team'})
  fourth_down_def_data = fourth_down_data.groupby('defteam')['fourth_down_converted'].mean().reset_index().rename(columns={'fourth_down_converted':'fourth_down_def', 'defteam':'team'})
  third_down_conv_data = pd.merge(third_down_off_data, third_down_def_data, on='team')
  fourth_down_conv_data = pd.merge(fourth_down_off_data, fourth_down_def_data, on='team')
  fullteamdata = fullteamdata.merge(third_down_conv_data, on='team')
  fullteamdata = fullteamdata.merge(fourth_down_conv_data, on='team')
  fullteamdata
  #4q/clutch
  third_down_4q_data = third_down_data[(third_down_data['qtr'] == 4) & (abs(third_down_data['score_differential']) <= 8)]
  fourth_down_4q_data = fourth_down_data[(fourth_down_data['qtr'] == 4) & (abs(fourth_down_data['score_differential']) <= 8)]

  third_down_clutch_data = third_down_data[(third_down_data['game_seconds_remaining'] <= 300) & (abs(third_down_data['score_differential']) <= 8)]
  fourth_down_clutch_data = fourth_down_data[(fourth_down_data['game_seconds_remaining'] <= 300) & (abs(fourth_down_data['score_differential']) <= 8)]
  fullteamdata['third_down_off_4q'] = third_down_4q_data.groupby('posteam')['third_down_converted'].mean().reset_index()['third_down_converted']
  fullteamdata['third_down_def_4q'] = third_down_4q_data.groupby('defteam')['third_down_converted'].mean().reset_index()['third_down_converted']
  fullteamdata['third_down_off_clutch'] = third_down_clutch_data.groupby('posteam')['third_down_converted'].mean().reset_index()['third_down_converted']
  fullteamdata['third_down_def_clutch'] = third_down_clutch_data.groupby('defteam')['third_down_converted'].mean().reset_index()['third_down_converted']
  #first down pass/run
  fullteamdata['first_down_passrun_off'] = (first_down_data.groupby('posteam').sum()['pass'] / first_down_data.groupby('posteam').sum()['rush']).reset_index()[0]
  fullteamdata['first_down_passrun_def'] = (first_down_data.groupby('defteam').sum()['pass'] / first_down_data.groupby('defteam').sum()['rush']).reset_index()[0]
  ### basic stats
  ##play_type
  #punt
  total_punt_data = pbp_clean[pbp_clean['play_type'] == 'punt'].groupby('posteam')['play_id'].count()
  fullteamdata['total_punt'] = total_punt_data.reset_index()['play_id']
  #goal_to_go
  fullteamdata['goal_to_go'] = pbp_clean.groupby('posteam')['goal_to_go'].sum().reset_index()['goal_to_go']
  fullteamdata['goal_to_go_def'] = pbp_clean.groupby('defteam')['goal_to_go'].sum().reset_index()['goal_to_go']

  #kickoffs
  fullteamdata['kickoff'] = pbp_clean[pbp_clean['play_type'] == 'kickoff'].groupby('defteam').count()['play_id'].reset_index()['play_id']
  fullteamdata['kickoff_received'] = pbp_clean[pbp_clean['play_type'] == 'kickoff'].groupby('posteam').count()['play_id'].reset_index()['play_id']
  #pass/run ratio
  fullteamdata['pass_run_ratio'] = (pbp_rp.groupby('posteam').sum()['pass'] / pbp_rp.groupby('posteam').sum()['rush']).reset_index()[0]
  fullteamdata['opp_pass_run_ratio'] = (pbp_rp.groupby('defteam').sum()['pass'] / pbp_rp.groupby('posteam').sum()['rush']).reset_index()[0]
  #number of plays
  playcount_data = pbp_clean.posteam.value_counts()
  fullteamdata['playcount'] = playcount_data.reset_index().posteam
  play_differential_data = playcount_data - pbp_clean.defteam.value_counts()
  fullteamdata['play_differential'] = play_differential_data.reset_index()[0]
  fullteamdata
  ##yardsperplay(off/def)
  fullteamdata['ydsperplay'] = pbp_rp.groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['ydsperattempt'] = pbp_rp[pbp_rp['pass'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['ydsperrush'] = pbp_rp[pbp_rp['rush'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperplay'] = pbp_rp.groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperattempt'] = pbp_rp[pbp_rp['pass'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperrush'] = pbp_rp[pbp_rp['rush'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  #4q
  fullteamdata['ydsperplay4q'] = fourthq.groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['ydsperattempt4q'] = fourthq[fourthq['pass'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['ydsperrush4q'] = fourthq[fourthq['rush'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperplay4q'] = fourthq.groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperattempt4q'] = fourthq[fourthq['pass'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperrush4q'] = fourthq[fourthq['rush'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  #clutch
  fullteamdata['ydsperplay_clutch'] = clutch_data.groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['ydsperattempt_clutch'] = clutch_data[clutch_data['pass'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['ydsperrush_clutch'] = clutch_data[clutch_data['rush'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperplay_clutch'] = clutch_data.groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperattempt_clutch'] = clutch_data[clutch_data['pass'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['opp_ydsperrush_clutch'] = clutch_data[clutch_data['rush'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  #YAC
  fullteamdata['off_YAC'] = pbp_rp.groupby('posteam')['yards_after_catch'].mean().reset_index()['yards_after_catch']
  fullteamdata['def_YAC'] = pbp_rp.groupby('defteam')['yards_after_catch'].mean().reset_index()['yards_after_catch']
  #average point differential(including 4th quarter/clutch)
  fullteamdata['off_pointdifferential'] = pbp_clean.groupby('posteam')['score_differential'].mean().reset_index()['score_differential']
  fullteamdata['def_pointdifferential'] = (pbp_clean.groupby('defteam')['score_differential'].mean() * (-1)).reset_index()['score_differential']
  fullteamdata['off_pointdifferential_4q'] = fourthq.groupby('posteam')['score_differential'].mean().reset_index()['score_differential']
  fullteamdata['def_pointdifferential_4q'] = (fourthq.groupby('defteam')['score_differential'].mean() * (-1)).reset_index()['score_differential']
  fullteamdata['off_pointdifferential_clutch'] = clutch_data.groupby('posteam')['score_differential'].mean().reset_index()['score_differential']
  fullteamdata['def_pointdifferential_clutch'] = (clutch_data.groupby('defteam')['score_differential'].mean() * (-1)).reset_index()['score_differential']
  #completion percentage
  fullteamdata['off_comp_percentage'] = pbp_rp[pbp_rp['pass'] == 1].groupby('posteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata['def_comp_percentage'] = pbp_rp[pbp_rp['pass'] == 1].groupby('defteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata['off_comp_percentage_4q'] = fourthq[fourthq['pass'] == 1].groupby('posteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata['def_comp_percentage_4q'] = fourthq[fourthq['pass'] == 1].groupby('defteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata['off_comp_percentage_clutch'] = clutch_data[clutch_data['pass'] == 1].groupby('posteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata['def_comp_percentage_clutch'] = clutch_data[clutch_data['pass'] == 1].groupby('defteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata
  #interceptions
  fullteamdata['off_interception_rate'] = pbp_rp[pbp_rp['pass'] == 1].groupby('posteam')['interception'].mean().reset_index()['interception']
  fullteamdata['def_interception_rate'] = pbp_rp[pbp_rp['pass'] == 1].groupby('defteam')['interception'].mean().reset_index()['interception']
  fullteamdata['off_interception_rate_4q'] = fourthq[fourthq['pass'] == 1].groupby('posteam')['interception'].mean().reset_index()['interception']
  fullteamdata['def_interception_rate_4q'] = fourthq[fourthq['pass'] == 1].groupby('defteam')['interception'].mean().reset_index()['interception']
  #fumble
  fullteamdata['off_fumble'] = pbp_clean.groupby('posteam')['fumble'].sum().reset_index()['fumble']
  fullteamdata['off_fumble_lost'] = pbp_clean.groupby('posteam')['fumble_lost'].sum().reset_index()['fumble_lost']
  fullteamdata['def_fumble_lost'] = pbp_clean.groupby('defteam')['fumble_lost'].sum().reset_index()['fumble_lost']
  fullteamdata['fumble_forced'] = pbp_clean.groupby('defteam')['fumble_forced'].sum().reset_index()['fumble_forced']
  #touchdown
  fullteamdata['touchdown_diff'] = (pbp_clean.groupby('posteam')['touchdown'].sum() - pbp_clean.groupby('defteam')['touchdown'].sum()).reset_index()['touchdown']
  fullteamdata['touchdown_diff_4q'] = (fourthq.groupby('posteam')['touchdown'].sum() - fourthq.groupby('defteam')['touchdown'].sum()).reset_index()['touchdown']
  fullteamdata['touchdownrate_off'] = (pbp_clean.groupby('posteam')['touchdown'].mean()).reset_index()['touchdown']
  fullteamdata['touchdownrate_def'] = (pbp_clean.groupby('defteam')['touchdown'].mean()).reset_index()['touchdown']
  #sack
  fullteamdata['sack_diff'] = (pbp_clean.groupby('defteam')['sack'].sum()-pbp_clean.groupby('posteam')['sack'].sum()).reset_index()['sack']
  fullteamdata['sack_diff4q'] = (fourthq.groupby('defteam')['sack'].sum()-fourthq.groupby('posteam')['sack'].sum()).reset_index()['sack']
  fullteamdata['sack_rate_pbp'] = pbp_rp.groupby('defteam')['sack'].mean().reset_index()['sack']
  fullteamdata['sackrate_def'] = pbp_rp[pbp_rp['pass']==1].groupby('defteam')['sack'].mean().reset_index()['sack']
  fullteamdata['sackrate_off'] = pbp_rp[pbp_rp['pass']==1].groupby('posteam')['sack'].mean().reset_index()['sack']
  #qbhit
  fullteamdata['qbhit_diff'] = (pbp_clean.groupby('defteam')['qb_hit'].sum()-pbp_clean.groupby('posteam')['qb_hit'].sum()).reset_index()['qb_hit']
  fullteamdata['qbhit_diff4q'] = (fourthq.groupby('defteam')['qb_hit'].sum()-fourthq.groupby('posteam')['qb_hit'].sum()).reset_index()['qb_hit']
  fullteamdata['qbhit_rate_pbp'] = pbp_rp.groupby('defteam')['qb_hit'].mean().reset_index()['qb_hit']
  fullteamdata['qbhitrate_def'] = pbp_rp[pbp_rp['pass']==1].groupby('defteam')['qb_hit'].mean().reset_index()['qb_hit']
  fullteamdata['qbhitrate_off'] = pbp_rp[pbp_rp['pass']==1].groupby('posteam')['qb_hit'].mean().reset_index()['qb_hit']
  ###advanced stats
  #win probability
  fullteamdata['off_winprob'] = pbp_clean.groupby('posteam')['wp'].mean().reset_index()['wp']
  fullteamdata['def_winprob'] = pbp_clean.groupby('defteam')['def_wp'].mean().reset_index()['def_wp']
  fullteamdata['off_winprob_fourthq'] = fourthq.groupby('posteam')['wp'].mean().reset_index()['wp']
  fullteamdata['def_winprob_fourthq'] = fourthq.groupby('defteam')['def_wp'].mean().reset_index()['def_wp']
  fullteamdata['off_winprob_clutch'] = clutch_data.groupby('posteam')['wp'].mean().reset_index()['wp']
  fullteamdata['def_winprob_clutch'] = clutch_data.groupby('defteam')['def_wp'].mean().reset_index()['def_wp']
  #series success
  fullteamdata['off_series_success'] = pbp_clean.groupby('posteam')['series_success'].mean().reset_index()['series_success']
  fullteamdata['def_series_success'] = pbp_clean.groupby('defteam')['series_success'].mean().reset_index()['series_success']
  fullteamdata['off_series_success_fourthq'] = fourthq.groupby('posteam')['series_success'].mean().reset_index()['series_success']
  fullteamdata['def_series_success_fourthq'] = fourthq.groupby('defteam')['series_success'].mean().reset_index()['series_success']
  fullteamdata['off_series_success_clutch'] = clutch_data.groupby('posteam')['series_success'].mean().reset_index()['series_success']
  fullteamdata['def_series_success_clutch'] = clutch_data.groupby('defteam')['series_success'].mean().reset_index()['series_success']
  clutch_data.groupby('defteam')['series_success'].mean().reset_index()
  #first down
  fullteamdata['firstdown_diff'] = (pbp_clean.groupby('posteam')['first_down'].sum()-pbp_clean.groupby('defteam')['first_down'].sum()).reset_index()['first_down']
  fullteamdata['firstdownrate_off'] = pbp_clean.groupby('posteam')['first_down'].mean().reset_index()['first_down']
  fullteamdata['firstdownrate_def'] = pbp_clean.groupby('defteam')['first_down'].mean().reset_index()['first_down']
  fullteamdata['firstdownrate_diff4q'] = (fourthq.groupby('posteam')['first_down'].mean()-fourthq.groupby('defteam')['first_down'].mean()).reset_index()['first_down']
  fullteamdata['firstdownrate_diffclutch'] = (clutch_data.groupby('posteam')['first_down'].mean()-clutch_data.groupby('defteam')['first_down'].mean()).reset_index()['first_down']
  #redzone/goaltogo
  fullteamdata['red_zone_off'] = (drive_data[(drive_data['drive_inside20']==1)&(drive_data['fixed_drive_result']=='Touchdown')].groupby('posteam')['play_id'].count() /drive_data.groupby('posteam')['drive_inside20'].sum()).reset_index()[0]
  fullteamdata['red_zone_def'] = (drive_data[(drive_data['drive_inside20']==1)&(drive_data['fixed_drive_result']=='Touchdown')].groupby('defteam')['play_id'].count()/drive_data.groupby('defteam')['drive_inside20'].sum()).reset_index()[0]
  fullteamdata['goaltogo_off'] = pbp_clean[pbp_clean['goal_to_go']==1].groupby('posteam').series_success.mean().reset_index()['series_success']
  fullteamdata['goaltogo_def'] = pbp_clean[pbp_clean['goal_to_go']==1].groupby('defteam').series_success.mean().reset_index()['series_success']
  ###special teams
  #punt_block_difference
  punt_block_data = (pbp_clean.groupby('posteam').punt_blocked.sum() - pbp_clean.groupby('defteam').punt_blocked.sum()).reset_index()\
  .rename(columns={'posteam':'team', 'punt_blocked':'punt_blocked_diff'})
  fullteamdata['punt_block'] = punt_block_data.punt_blocked_diff

  #punt_inside_20 proportion
  punt_inside20_data = (pbp_clean[pbp_clean['play_type'] == 'punt'].groupby('posteam')['punt_inside_twenty'].sum() / total_punt_data).reset_index()\
  .rename(columns={'posteam':'team', 0:'punt_inside_twenty'})
  fullteamdata['punt_inside20'] = punt_inside20_data.punt_inside_twenty
  fullteamdata
  #avg starting field position (exclude kickoffs)
  drive_nokickoff_data = drive_data[drive_data['play_type']!='kickoff']
  drive_avgstartoff_data = drive_nokickoff_data.groupby('posteam')[['yardline_100']].mean().reset_index()
  fullteamdata['drive_avgstart_off'] = drive_avgstartoff_data['yardline_100']
  drive_avgstartdef_data = drive_nokickoff_data.groupby('defteam')[['yardline_100']].mean().reset_index()
  fullteamdata['drive_avgstart_def'] = drive_avgstartdef_data['yardline_100']
  #avg field position
  avgfieldposoff_data = pbp_clean.groupby('posteam')[['yardline_100']].mean().reset_index()
  avgfieldposdef_data = pbp_clean.groupby('defteam')[['yardline_100']].mean().reset_index()
  fullteamdata['avg_fieldpos_off'] = avgfieldposoff_data['yardline_100']
  fullteamdata['avg_fieldpos_def'] = avgfieldposdef_data['yardline_100']
  #field goals
  field_goal_data = (drive_data[drive_data['fixed_drive_result'] == 'Field goal'].groupby('posteam').count()['play_id'] / pbp_clean.groupby('posteam')['field_goal_attempt'].sum()).reset_index()
  fullteamdata['field_goal'] = field_goal_data[0]
  ###penalties
  penalties_data = pbp_temp.groupby('penalty_team').count()['play_id'].reset_index()
  fullteamdata['penalties'] = penalties_data['play_id']
  penaltyyards_data = pbp_temp.groupby('penalty_team').sum()['penalty_yards'].reset_index()
  fullteamdata['penalty_yards'] = penaltyyards_data['penalty_yards']
  #third/fourth down
  penalties_3rdor4thdown_data = pbp_temp[(pbp_temp['down'] == 3) | (pbp_temp['down'] == 4)].groupby('penalty_team').count()['play_id'].reset_index()
  fullteamdata['penalties_bigdowns'] = penalties_3rdor4thdown_data['play_id']
  penaltyyards_3rdor4thdown_data = pbp_temp[(pbp_temp['down'] == 3) | (pbp_temp['down'] == 4)].groupby('penalty_team').sum()['penalty_yards'].reset_index()
  fullteamdata['penaltyyards_bigdowns'] = penaltyyards_3rdor4thdown_data['penalty_yards']
  ### per drive
  #drivecount
  off_drive_count_data = drive_data.groupby('posteam')['play_id'].count().reset_index().rename(columns={'play_id':'drive_count', 'posteam':'team'})
  def_drive_count_data = drive_data.groupby('defteam')['play_id'].count().reset_index().rename(columns={'play_id':'drive_count', 'posteam':'team'})
  off_fourthquarter_drivecount_data = fourth_quarter_drives.groupby('posteam')['play_id'].count().reset_index()
  def_fourthquarter_drivecount_data = fourth_quarter_drives.groupby('defteam')['play_id'].count().reset_index()
  fullteamdata['drive_count'] = off_drive_count_data['drive_count']
  #plays per drive, first downs per drive, drive inside 20, drive ended with score
  fullteamdata[['off_plays_perdrive','off_firstdowns_perdrive','off_drive_inside20','off_drive_endedwithscore']] = drive_data.groupby('posteam')[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']].mean().reset_index()[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']]
  fullteamdata[['def_plays_perdrive','def_firstdowns_perdrive','def_drive_inside20','def_drive_endedwithscore']] = drive_data.groupby('defteam')[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']].mean().reset_index()[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']]
  fullteamdata[['off_plays_perdrive_4q','off_firstdowns_perdrive_4q','off_drive_inside20_4q','off_drive_endedwithscore_4q']] = drive_data[drive_data['qtr']==4].groupby('posteam')[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']].mean().reset_index()[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']]
  fullteamdata[['def_plays_perdrive_4q','def_firstdowns_perdrive_4q','def_drive_inside20_4q','def_drive_endedwithscore_4q']] = drive_data[drive_data['qtr']==4].groupby('defteam')[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']].mean().reset_index()[['drive_play_count','drive_first_downs','drive_inside20','drive_ended_with_score']]
  #yards per drive
  fullteamdata['off_yards_per_drive'] = (pbp_clean.groupby('posteam')['yards_gained'].sum().reset_index()['yards_gained']/off_drive_count_data['drive_count']).reset_index()[0]
  fullteamdata['off_yards_per_drive_4q'] = (fourthq.groupby('posteam')['yards_gained'].sum().reset_index()['yards_gained']/off_fourthquarter_drivecount_data['play_id']).reset_index()[0]
  fullteamdata['def_yards_per_drive'] = (pbp_clean.groupby('defteam')['yards_gained'].sum().reset_index()['yards_gained']/def_drive_count_data['drive_count']).reset_index()[0]
  fullteamdata['def_yards_per_drive4q'] = (fourthq.groupby('defteam')['yards_gained'].sum().reset_index()['yards_gained']/def_fourthquarter_drivecount_data['play_id']).reset_index()[0]
  #touchdowns per drive
  fullteamdata['off_touchdowns_perdrive'] = (pbp_clean.groupby('posteam')['touchdown'].sum().reset_index()['touchdown']/off_drive_count_data['drive_count']).reset_index()[0]
  fullteamdata['def_touchdowns_perdrive'] = (pbp_clean.groupby('defteam')['touchdown'].sum().reset_index()['touchdown']/def_drive_count_data['drive_count']).reset_index()[0]
  fullteamdata['off_touchdowns_perdrive_4q'] = (fourthq.groupby('posteam')['touchdown'].sum().reset_index()['touchdown']/off_fourthquarter_drivecount_data['play_id']).reset_index()[0]
  fullteamdata['def_touchdowns_perdrive_4q'] = (fourthq.groupby('defteam')['touchdown'].sum().reset_index()['touchdown']/def_fourthquarter_drivecount_data['play_id']).reset_index()[0]
  #unsuccessful drives per drive
  fullteamdata['off_bad_drives_perdrive'] = (drive_data[(drive_data['fixed_drive_result']!='Touchdown') & (drive_data['fixed_drive_result']!='Field Goal')].groupby('posteam')['play_id'].count().reset_index()['play_id']/off_drive_count_data['drive_count'])
  fullteamdata['def_bad_drives_perdrive'] = (drive_data[(drive_data['fixed_drive_result']!='Touchdown') & (drive_data['fixed_drive_result']!='Field Goal')].groupby('defteam')['play_id'].count().reset_index()['play_id']/def_drive_count_data['drive_count'])
  fullteamdata['off_bad_drives_perdrive_4q'] = (fourth_quarter_drives[(fourth_quarter_drives['fixed_drive_result']!='Touchdown') & (fourth_quarter_drives['fixed_drive_result']!='Field Goal')].groupby('posteam')['play_id'].count().reset_index()['play_id']/off_fourthquarter_drivecount_data['play_id'])
  fullteamdata['def_bad_drives_perdrive_4q'] = (fourth_quarter_drives[(fourth_quarter_drives['fixed_drive_result']!='Touchdown') & (fourth_quarter_drives['fixed_drive_result']!='Field Goal')].groupby('defteam')['play_id'].count().reset_index()['play_id']/def_fourthquarter_drivecount_data['play_id'])

  ###seasonal dataframes (per game and per drive stats)
  drive_count_data = drive_data.groupby('posteam')['play_id'].count().reset_index().rename(columns={'play_id':'drive_count', 'posteam':'team'})
  game_count_data = win_totals_temp.groupby('home_team')['week'].nunique() + win_totals_temp.groupby('away_team')['week'].nunique()
  one_score_data = win_totals_temp[abs(win_totals_temp['result']) <=8]
  reg_data = win_totals_temp[win_totals_temp['game_type']=='REG']
  groupby_home = win_totals_temp.groupby('home_team')
  groupby_away = win_totals_temp.groupby('away_team')
  groupby_home_onescore = one_score_data.groupby('home_team')
  groupby_away_onescore = one_score_data.groupby('away_team')
  groupby_home_reg = reg_data.groupby('home_team')
  groupby_away_reg = reg_data.groupby('away_team')
  reggame_count_data = groupby_home_reg['week'].nunique()+groupby_away_reg['week'].nunique()

  #point differential
  fullteamdata['pointdifferential'] = (win_totals_temp.groupby('home_team')['result'].sum() + (win_totals_temp.groupby('away_team')['result'].sum()*-1)).reset_index()['result']
  fullteamdata['pointdifferential_onescore'] = (one_score_data.groupby('home_team')['result'].sum() + (one_score_data.groupby('away_team')['result'].sum()*-1)).reset_index()['result']
  #win percentage
  fullteamdata['win_percentage']=wins_data['wins']/((groupby_home['week'].nunique() + groupby_away['week'].nunique()).reset_index()['week'])
  fullteamdata['win_percentage_reg']=((groupby_home_reg['home_win'].sum()+groupby_away_reg['away_win'].sum())/\
  ((groupby_home_reg['week'].nunique() + groupby_away_reg['week'].nunique()))).reset_index()[0]
  fullteamdata['win_percentage_onescore']=((groupby_home_onescore['home_win'].sum()+groupby_away_onescore['away_win'].sum())/(groupby_home_onescore['week'].nunique()+groupby_away_onescore['week'].nunique())).reset_index()[0]
  fullteamdata
  #spread line
  fullteamdata['avg_spreadline'] = (groupby_home['spread_line'].mean()-groupby_away['spread_line'].mean()).reset_index()['spread_line']
  #points per game
  fullteamdata['points_pergame_off'] = ((groupby_home['home_score'].sum()+groupby_away['away_score'].sum())/game_count_data).reset_index()[0]
  fullteamdata['points_pergame_def'] = ((groupby_home['away_score'].sum()+groupby_away['home_score'].sum())/game_count_data).reset_index()[0]
  fullteamdata['points_pergame_off_reg'] = ((groupby_home_reg['home_score'].sum()+groupby_away_reg['away_score'].sum())/reggame_count_data).reset_index()[0]
  fullteamdata['points_pergame_def_reg'] = ((groupby_home_reg['away_score'].sum()+groupby_away_reg['home_score'].sum())/reggame_count_data).reset_index()[0]
  #yards per game
  fullteamdata['yds_pergame_off'] = (pbp_clean.groupby('posteam')['yards_gained'].sum()/game_count_data).reset_index()[0]
  fullteamdata['yds_pergame_def'] = (pbp_clean.groupby('defteam')['yards_gained'].sum()/game_count_data).reset_index()[0]
  fullteamdata['yds_pergame_off_reg'] = (pbp_clean[pbp_clean['season_type']=='REG'].groupby('posteam')['yards_gained'].sum()/reggame_count_data).reset_index()[0]
  fullteamdata['yds_pergame_def_reg'] = (pbp_clean[pbp_clean['season_type']=='REG'].groupby('defteam')['yards_gained'].sum()/reggame_count_data).reset_index()[0]
  fullteamdata
  #first downs per game
  fullteamdata['firstdowns_pergame_off'] = (pbp_clean.groupby('posteam')['first_down'].sum()/game_count_data).reset_index()[0]
  fullteamdata['firstdowns_pergame_def'] = (pbp_clean.groupby('defteam')['first_down'].sum()/game_count_data).reset_index()[0]
  fullteamdata['firstdowns_pergame_off_reg'] = (pbp_clean[pbp_clean['season_type']=='REG'].groupby('posteam')['first_down'].sum()/reggame_count_data).reset_index()[0]
  fullteamdata['firstdowns_pergame_def_reg'] = (pbp_clean[pbp_clean['season_type']=='REG'].groupby('defteam')['first_down'].sum()/reggame_count_data).reset_index()[0]
  ###post-thanksgiving stats
  post_thanksgiving = pbp_clean[pbp_clean['week']>=13]
  post_thanksgiving_4q = fourthq[pbp_clean['week']>=13]
  groupby_thanksgiving_home = win_totals_temp[win_totals_temp['week']>=13].groupby('home_team')
  groupby_thanksgiving_away = win_totals_temp[win_totals_temp['week']>=13].groupby('away_team')
  game_count_thanksgiving = groupby_thanksgiving_home['week'].nunique()+groupby_thanksgiving_away['week'].nunique()
  #yards per play(including fourth quarter)
  fullteamdata['postthxgiving_ydsperplay'] = post_thanksgiving.groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['postthxgiving_ydsperattempt'] = post_thanksgiving[post_thanksgiving['pass'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['postthxgiving_ydsperrush'] = post_thanksgiving[post_thanksgiving['rush'] == 1].groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['postthxgiving_opp_ydsperplay'] = post_thanksgiving.groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['postthxgiving_opp_ydsperattempt'] = post_thanksgiving[post_thanksgiving['pass'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['postthxgiving_opp_ydsperrush'] = post_thanksgiving[post_thanksgiving['rush'] == 1].groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']
  #4q
  fullteamdata['postthxgiving_ydsperplay4q'] = post_thanksgiving_4q.groupby('posteam')['yards_gained'].mean().reset_index()['yards_gained']
  fullteamdata['postthxgiving_opp_ydsperplay4q'] = post_thanksgiving_4q.groupby('defteam')['yards_gained'].mean().reset_index()['yards_gained']

  #epa and wpa(including 4q)
  fullteamdata[['postthxgiving_epa_off','postthxgiving_wpa_off']] = post_thanksgiving.groupby('posteam')[['epa','wpa']].mean().reset_index()[['epa','wpa']]
  fullteamdata[['postthxgiving_epa_def','postthxgiving_wpa_def']] = def_epa_data = post_thanksgiving.groupby('defteam')[['epa', 'wpa']].mean().reset_index()[['epa','wpa']]
  fullteamdata[['postthxgiving_epa_off_4q','postthxgiving_wpa_off_4q']] = post_thanksgiving_4q.groupby("posteam")[['epa', 'wpa']].mean().reset_index()[['epa','wpa']]
  fullteamdata[['postthxgiving_epa_def_4q','postthxgiving_wpa_def_4q']] = post_thanksgiving_4q.groupby("defteam")[['epa', 'wpa']].mean().reset_index()[['epa','wpa']]
  #down stats
  postthanksgiving_seconddown = post_thanksgiving[post_thanksgiving['down'] == 2]
  postthanksgiving_thirddown = post_thanksgiving[(post_thanksgiving['down'] == 3)]
  fullteamdata['postthxgiving_2ndydstogo_off'] = postthanksgiving_seconddown.groupby('posteam')['ydstogo'].mean().reset_index()['ydstogo']
  fullteamdata['postthxgiving_2ndydstogo_def'] = postthanksgiving_seconddown.groupby('defteam')['ydstogo'].mean().reset_index()['ydstogo']
  fullteamdata['postthxgiving_3rdydstogo_off'] = postthanksgiving_thirddown.groupby('posteam')['ydstogo'].mean().reset_index()['ydstogo']
  fullteamdata['postthxgiving_3rdydstogo_def'] = postthanksgiving_thirddown.groupby('defteam')['ydstogo'].mean().reset_index()['ydstogo']
  fullteamdata['postthxgiving_3rddownconverted_off'] = postthanksgiving_thirddown.groupby('posteam')['third_down_converted'].mean().reset_index()['third_down_converted']
  fullteamdata['postthxgiving_3rddownconverted_def'] = postthanksgiving_thirddown.groupby('defteam')['third_down_converted'].mean().reset_index()['third_down_converted']
  #points per game
  fullteamdata['postthxgiving_points_pergame_off'] = ((groupby_thanksgiving_home['home_score'].sum()+groupby_thanksgiving_away['away_score'].sum())/game_count_thanksgiving).reset_index()[0]
  fullteamdata['postthxgiving_points_pergame_def'] = ((groupby_thanksgiving_home['away_score'].sum()+groupby_thanksgiving_away['home_score'].sum())/game_count_thanksgiving).reset_index()[0]
  #yards per game
  fullteamdata['postthxgiving_yards_pergame_off'] = ((post_thanksgiving.groupby('posteam')['yards_gained'].sum())/game_count_thanksgiving).reset_index()[0]
  fullteamdata['postthxgiving_yards_pergame_def'] = ((post_thanksgiving.groupby('defteam')['yards_gained'].sum())/game_count_thanksgiving).reset_index()[0]
  #win percentage
  fullteamdata['postthxgiving_win_percentage']=((groupby_thanksgiving_home['home_win'].sum()+groupby_thanksgiving_away['away_win'].sum())/game_count_thanksgiving).reset_index()[0]
  #average point differential
  fullteamdata['postthxgiving_off_pointdifferential'] = post_thanksgiving.groupby('posteam')['score_differential'].mean().reset_index()['score_differential']
  fullteamdata['postthxgiving_def_pointdifferential'] = (post_thanksgiving.groupby('defteam')['score_differential'].mean() * (-1)).reset_index()['score_differential']
  #completion percentage
  fullteamdata['postthxgiving_off_comp_percentage'] = post_thanksgiving[post_thanksgiving['pass'] == 1].groupby('posteam')['complete_pass'].mean().reset_index()['complete_pass']
  fullteamdata['postthxgiving_def_comp_percentage'] = post_thanksgiving[post_thanksgiving['pass'] == 1].groupby('defteam')['complete_pass'].mean().reset_index()['complete_pass']
  ###new section
  win_totals_2022 = win_totals[win_totals['season']==i+1]
  seasonal_temp = seasonal_data[seasonal_data['season']==i]
  if i == 2023:
    depth_chart_temp = depth_chart[depth_chart['season']==2023]
  else:
    depth_chart_temp = depth_chart[depth_chart['season']==i+1]

  roster_temp = roster[(roster['season']==i)|(roster['season']==i+1)]
  qb1 = depth_chart_temp[(depth_chart_temp['depth_position']=='QB')&(depth_chart_temp['depth_team']=='1')&(depth_chart_temp['week']==1)][['club_code','gsis_id','full_name']]
  qb1 = qb1.drop_duplicates()
  #check for rookies
  qb1_transpose = qb1
  qb1_transpose=qb1_transpose.set_index('club_code')
  qb1_transpose = qb1_transpose.transpose()
  count=0
  for a in qb1_transpose:
    count+=1
    if qb1_transpose.loc['gsis_id',a] not in list(roster[roster['season']==i]['player_id']):
      #set chad henne as default
      qb1_transpose[a] = ['00-0026625','Brian Hoyer']
  qb1_transpose=qb1_transpose.transpose()
  qb1_seasonstats = seasonal_temp.set_index('player_id').filter(qb1_transpose['gsis_id'], axis=0)
  qb1.set_index('gsis_id',inplace=True)
  qb1_seasonstats['2022_team'] = qb1['club_code']
  qb1_seasonstats = qb1_seasonstats.reset_index().set_index('2022_team')
  qb1_seasonstats.sort_index(inplace=True)
  qb1_seasonstats = qb1_seasonstats.transpose()
  #fill in default stats
  for b in range(0,32):
    if qb1_seasonstats.iloc[4, b] < 75:
      qb1_seasonstats.iloc[4,b]=90
      qb1_seasonstats.iloc[3,b]=53
      qb1_seasonstats.iloc[5,b]=560
      qb1_seasonstats.iloc[6,b]=5
      qb1_seasonstats.iloc[7,b]=2
      qb1_seasonstats.iloc[15,b]=1.0
  qb1_seasonstats=qb1_seasonstats.transpose()
  ##account for rookies!!!
  #epa/db difference

  fullteamdata['epa/db_diff'] = ((qb1_seasonstats['passing_epa']/qb1_seasonstats['attempts']) - pbp_rp[pbp_rp['pass']==1].groupby('posteam')['epa'].mean()).reset_index()[0]


  qb1_seasonstats.reset_index(inplace=True)
  #completion percentage
  fullteamdata['comppct_diff'] = ((qb1_seasonstats['completions']/qb1_seasonstats['attempts']) - fullteamdata['off_comp_percentage']).reset_index()[0]
  #pass touchdowns minus interceptions
  fullteamdata['td-int_diff'] = ((qb1_seasonstats['passing_tds']-qb1_seasonstats['interceptions']) - (pbp_rp[pbp_rp['pass']==1].groupby('posteam')['touchdown'].sum()-\
                                      pbp_rp[pbp_rp['pass']==1].groupby('posteam')['interception'].sum()).reset_index()[0]).reset_index()[0]
  # yards per attempt
  fullteamdata['ypa_diff'] = ((qb1_seasonstats['passing_yards']/qb1_seasonstats['attempts']) - (pbp_rp[pbp_rp['pass']==1].groupby('posteam')['yards_gained'].mean()).reset_index()['yards_gained']).reset_index()[0]
  #fill in default data for qbs who got few snaps last season
  nfelo_wintotals = ''
  if i == 2019:
    nfelo_wintotals = pd.read_csv("nfl-win-totals_2020.csv")
  if i == 2020:
    nfelo_wintotals = pd.read_csv("nfl-win-totals_2021.csv")
  if i == 2021:
    nfelo_wintotals = pd.read_csv("nfl-win-totals_2022.csv")
  if i == 2022:
    nfelo_wintotals = pd.read_csv("nfl-win-totals_2023.csv")
  if i == 2023:
    nfelo_wintotals = pd.read_csv("nfl-win-totals_2024.csv")
  nfelo_wintotals = nfelo_wintotals[['team','line','line_adj','over_odds','under_odds','wt_rating','sos_projected']].sort_values(by='team').reset_index(drop=True)
  nfelo_wintotals=nfelo_wintotals.transpose()
  #swap lac and lar so that it is consistent with the var fullteamdata
  lac = nfelo_wintotals[16]
  nfelo_wintotals[16] = nfelo_wintotals[17]
  nfelo_wintotals[17]=lac
  nfelo_wintotals=nfelo_wintotals.transpose()
  nfelo_wintotals
  fullteamdata = fullteamdata.join(nfelo_wintotals.iloc[:,1:])
  #average experience
  fullteamdata['avg_exp_2021'] = (roster[roster['season']==i].groupby('team')['years_exp'].mean()).reset_index()['years_exp']
  fullteamdata['avg_exp_2022'] = roster[roster['season']==i+1].groupby('team')['years_exp'].mean().reset_index()['years_exp']
  fullteamdata['exp_diff'] = fullteamdata['avg_exp_2021']-fullteamdata['avg_exp_2022']
  #wins
  fullteamdata['wins'] = wins_data['wins']

# fullteamdata_2021
# depth_chart_temp = depth_chart[depth_chart['season']==2022]
# roster_temp = roster[roster['season']==2022]
# qb1 = depth_chart_temp[(depth_chart_temp['position']=='QB')&(depth_chart_temp['depth_team']=='1')&(depth_chart_temp['week']==1)][['club_code','gsis_id','full_name']]
# #check for rookies
# qb1_transpose = qb1
# qb1_transpose=qb1_transpose.set_index('club_code')
# qb1_transpose = qb1_transpose.transpose()

# for i in qb1_transpose:
#   if qb1_transpose.loc['gsis_id',i] not in list(roster_temp['player_id']):
#     #set chad henne as default
#     qb1_transpose[i] = ['00-0026197','Chad Henne']
# qb1_transpose
# qb1_transpose=qb1_transpose.transpose()
# qb1_seasonstats = seasonal_temp.set_index('player_id').filter(qb1_transpose['gsis_id'], axis=0)
# qb1.set_index('gsis_id',inplace=True)
# qb1_seasonstats['2022_team'] = qb1['club_code']
# qb1_seasonstats = qb1_seasonstats.reset_index().set_index('2022_team')
# qb1_seasonstats.sort_index(inplace=True)
# qb1_seasonstats = qb1_seasonstats.transpose()
# for i in range(0,32):
#     if qb1_seasonstats.iloc[4, i] < 75:
#       qb1_seasonstats.iloc[4,i]=90
#       qb1_seasonstats.iloc[3,i]=53
#       qb1_seasonstats.iloc[5,i]=560
#       qb1_seasonstats.iloc[6,i]=5
#       qb1_seasonstats.iloc[7,i]=2
#       qb1_seasonstats.iloc[15,i]=1.0

# qb1_transpose.drop_duplicates().shape
# seasonal_temp
  fullteamdata_arr.append(fullteamdata)
# fullteamdata
# for i in fullteamdata:
#   i.dropna()
#   print(i.shape)
fullteamdata
# win_totals_2021.groupby('home_team')['week'].nunique() + win_totals_2021.groupby('away_team')['week'].nunique()

<ipython-input-8-08f355a02be1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  win_totals_temp.replace(to_replace='OAK',value='LV',inplace=True)
<ipython-input-8-08f355a02be1>:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fourth_quarter_drives = drive_data[(drive_data['qtr']==4)&(abs(pbp_clean['score_differential']) <= 8 )]
<ipython-input-8-08f355a02be1>:101: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  fullteamdata['first_down_passrun_off'] = (first_down_data.groupby('posteam').sum()['pass'] / first_down_data.groupby('posteam').sum()['rush']).reset_index()[0]
<ipython-input-

,team,off_epa,off_wpa,def_epa,def_wpa,offrp_epa,offrp_wpa,defrp_epa,defrp_wpa,off4min_epa,...,line,line_adj,over_odds,under_odds,wt_rating,sos_projected,avg_exp_2021,avg_exp_2022,exp_diff,wins
0,ARI,-0.042654,-0.000497,0.068829,0.002571,-0.071296,-0.001515,0.085271,0.002478,-0.019190,...,6.5,6.616764,-141,118,-2.467977,0.754142,2.961165,3.168317,-0.207152,4
1,ATL,-0.089635,-0.000454,-0.036285,0.000831,-0.098896,-0.001409,-0.059397,-0.000518,-0.089301,...,9.5,9.627491,-144,121,0.642931,-1.374274,3.227723,3.623853,-0.396130,7
2,BAL,0.062749,0.001371,-0.117852,-0.001606,0.062820,0.001560,-0.142003,-0.002625,0.123893,...,11.5,11.187048,180,-218,5.631127,0.861881,3.494624,3.258427,0.236197,14
3,BUF,0.082664,0.002718,-0.044011,0.001034,0.114988,0.003625,-0.065522,-0.000182,0.064779,...,10.5,10.275167,149,-179,4.327087,1.168521,4.242105,3.563830,0.678275,12
4,CAR,-0.140244,-0.002974,0.032772,0.001884,-0.176669,-0.004343,0.022974,0.000935,-0.220968,...,4.5,4.88991,-260,212,-6.773102,-0.27432,3.461538,3.414414,0.047124,2
5,CHI,-0.047021,-0.000336,-0.047997,0.000191,-0.048790,-0.000882,-0.041494,0.000540,-0.267124,...,8.5,8.737178,-184,153,0.565341,0.070915,3.141176,3.315217,-0.174041,7
6,CIN,-0.005874,0.002001,0.023929,0.002066,-0.010156,0.001648,0.033753,0.001469,-0.085121,...,10.5,10.39197,116,-138,3.25925,-0.138614,2.602273,2.564356,0.037916,9
7,CLE,-0.115853,-0.001647,-0.149549,-0.003547,-0.150273,-0.003123,-0.169176,-0.003407,-0.029510,...,8.5,8.721935,-178,148,1.428758,1.013027,3.653465,3.539823,0.113642,11
8,DAL,0.099210,0.002526,-0.065227,0.001009,0.102504,0.002551,-0.084326,0.000400,0.104535,...,10.5,10.323845,134,-161,3.947053,0.605355,3.250000,3.420000,-0.170000,12
9,DEN,-0.049036,-0.000604,0.015174,0.000456,-0.058099,-0.001190,0.019246,0.000304,0.025076,...,5.5,5.671493,-157,134,-4.656013,0.355086,3.054945,2.915663,0.139282,8


In [9]:

def transform(df_temp):
  df_clean = df_temp[(df_temp['play_type'] != 'qb_kneel') & (df_temp['play_type'] != 'no_play') & (df_temp['play_type'] != 'qb_spike')].dropna(subset=['play_type'])
  df = df_clean.groupby('posteam').mean()[['epa','wpa',]].rename(columns={'epa':'off_epa','wpa':'off_wpa'})
  df[['def_epa','def_wpa']] = df_clean.groupby('defteam').mean()[['epa','wpa',]]
  pbp_rp_transform = df_clean[(df_clean['pass']==1)|(df_clean['rush']==1)]
  second_down_transform = df_clean[df_clean['down']==2]
  third_down_transform = df_clean[df_clean['down']==3]
  df[['offrp_epa','offrp_wpa']] = (pbp_rp_transform.groupby('posteam').mean()[['epa','wpa',]])
  df[['defrp_epa','defrp_wpa']] = (pbp_rp_transform.groupby('defteam').mean()[['epa','wpa',]])
  df['ydstogo_2nd_off'] = (second_down_transform.groupby('posteam').mean()[['ydstogo']])
  df['ydstogo2nd_def'] = (second_down_transform.groupby('defteam').mean()[['ydstogo']])
  df['ydstogo_3rd_off'] = (third_down_transform.groupby('posteam').mean()[['ydstogo']])
  df['ydstogo_3rd_def'] = (third_down_transform.groupby('defteam').mean()[['ydstogo']])
  df['third_down_off'] = (third_down_transform.groupby('posteam').mean()[['third_down_converted']])
  df['third_down_def'] = (third_down_transform.groupby('defteam').mean()[['third_down_converted']])
  df['ydsperplay'] = (pbp_rp_transform.groupby('posteam').mean()['yards_gained'])
  df['ydsperattempt'] = (pbp_rp_transform[pbp_rp_transform['pass']==1].groupby('posteam').mean()['yards_gained'])
  df['ydsperrush'] = (pbp_rp_transform[pbp_rp_transform['rush']==1].groupby('posteam').mean()['yards_gained'])
  df['opp_ydsperplay'] = (pbp_rp_transform.groupby('defteam').mean()['yards_gained'])
  df['opp_ydsperattempt'] = (pbp_rp_transform[pbp_rp_transform['pass']==1].groupby('defteam').mean()['yards_gained'])
  df['opp_ydsperrush'] = (pbp_rp_transform[pbp_rp_transform['rush']==1].groupby('defteam').mean()['yards_gained'])
  return df

# transform(pbp_2022[(pbp_2022['week']<5)&(pbp_2022['week']>=3)], 'TB','DAL')

# 'TB' not in list(win_totals_2022[win_totals_2022['week']==11]['away_team']) and \
#       'TB' not in list(win_totals_2022[win_totals_2022['week']==11]['home_team'])

In [10]:
###thisseason method
def thisseason(pbp_temp, schedule_temp):
  df_clean = pbp_temp[(pbp_temp['play_type'] != 'qb_kneel') & (pbp_temp['play_type'] != 'no_play') & (pbp_temp['play_type'] != 'qb_spike')].dropna(subset=['play_type'])
  pbp_rp_this = df_clean[(df_clean['pass']==1)|(df_clean['rush']==1)]
  second_down_this = df_clean[df_clean['down']==2]
  third_down_this = df_clean[df_clean['down']==3]
  fourq_this = df_clean[df_clean['qtr']==4]
  groupby_away = schedule_temp.groupby('away_team')
  groupby_home = schedule_temp.groupby('home_team')
  schedule_temp['away_win'] = schedule_temp.apply(lambda row : 0 if row['result']>0 else 1, axis=1)
  schedule_temp['home_win'] = schedule_temp.apply(lambda row : 1 if row['result']>0 else 0, axis=1)
  game_count_this = groupby_home['week'].nunique()+groupby_away['week'].nunique()
  #yards per play(including fourth quarter)
  df = df_clean.groupby('posteam')[['yards_gained']].mean().rename(columns={'yards_gained':'ydsperplay'})
  df['ydsperattempt'] = df_clean[df_clean['pass'] == 1].groupby('posteam')['yards_gained'].mean()
  df['ydsperrush'] = df_clean[df_clean['rush'] == 1].groupby('posteam')['yards_gained'].mean()
  df['opp_ydsperplay'] = df_clean.groupby('defteam')['yards_gained'].mean()
  df['opp_ydsperattempt'] = df_clean[df_clean['pass'] == 1].groupby('defteam')['yards_gained'].mean()
  df['opp_ydsperrush'] = df_clean[df_clean['rush'] == 1].groupby('defteam')['yards_gained'].mean()
  #4q
  df['ydsperplay4q'] = fourq_this.groupby('posteam')['yards_gained'].mean()
  df['opp_ydsperplay4q'] = fourq_this.groupby('defteam')['yards_gained'].mean()

  #epa and wpa(including 4q)
  df[['epa_off','wpa_off']] = df_clean.groupby('posteam')[['epa','wpa']].mean()
  df[['epa_def','wpa_def']] = df_clean.groupby('defteam')[['epa', 'wpa']].mean()
  df[['epa_off_4q','wpa_off_4q']] = fourq_this.groupby("posteam")[['epa', 'wpa']].mean()
  df[['epa_def_4q','wpa_def_4q']] = fourq_this.groupby("defteam")[['epa', 'wpa']].mean()
  #down stats
  df['2ndydstogo_off'] = second_down_this.groupby('posteam')['ydstogo'].mean()
  df['2ndydstogo_def'] = second_down_this.groupby('defteam')['ydstogo'].mean()
  df['3rdydstogo_off'] = third_down_this.groupby('posteam')['ydstogo'].mean()
  df['3rdydstogo_def'] = third_down_this.groupby('defteam')['ydstogo'].mean()
  df['3rddownconverted_off'] = third_down_this.groupby('posteam')['third_down_converted'].mean()
  df['3rddownconverted_def'] = third_down_this.groupby('defteam')['third_down_converted'].mean()
  #points per game
  df['points_pergame_off'] = ((groupby_home['home_score'].sum()+groupby_away['away_score'].sum())/game_count_this)
  df['points_pergame_def'] = ((groupby_home['away_score'].sum()+groupby_away['home_score'].sum())/game_count_this)
  #yards per game
  df['yards_pergame_off'] = ((df_clean.groupby('posteam')['yards_gained'].sum())/game_count_this)
  df['yards_pergame_def'] = ((df_clean.groupby('defteam')['yards_gained'].sum())/game_count_this)
  #win percentage
  df['win_percentage']=((groupby_home['home_win'].sum()+groupby_away['away_win'].sum())/game_count_this)
  #average point differential
  df['off_pointdifferential'] = df_clean.groupby('posteam')['score_differential'].mean()
  df['def_pointdifferential'] = (df_clean.groupby('defteam')['score_differential'].mean() * (-1))
  #completion percentage
  df['off_comp_percentage'] = df_clean[df_clean['pass'] == 1].groupby('posteam')['complete_pass'].mean()
  df['def_comp_percentage'] = df_clean[df_clean['pass'] == 1].groupby('defteam')['complete_pass'].mean()

  return df.add_suffix("_thisyear")


In [11]:
### organization, create an list of dataframes for each game


game_list_prev = []
results = []
this_season = fullteamdata_arr.pop(current_year - 2020)
year = 2020
count = 0
for fullteamdata in fullteamdata_arr:
  pbp_thisyear=pbp_data[pbp_data['season']==year]
  pbp_lastyear=pbp_data[pbp_data['season']==year-1]
  win_totals_temp = win_totals[win_totals['season']==year]
  week1_df = pd.DataFrame(index=fullteamdata.columns)
  fullteamdata.set_index('team',inplace=True)
  thanksgiving = fullteamdata.iloc[:,176:207]
  thanksgiving.columns = thanksgiving.columns.str.replace('postthxgiving_','')
  thanksgiving = thanksgiving.add_suffix('_thisyear')
  for i in range(1, 23, 1):
    if i <= 4:
      df_thisseason = thanksgiving
      df_temp=pbp_lastyear[pbp_lastyear['week']>7]
      df_past4weeks=transform(df_temp)
    else:
      df_thisseason = thisseason(pbp_thisyear[pbp_thisyear['week']<i], win_totals_temp[win_totals_temp['week']<i])
      df_temp=pbp_thisyear[(pbp_thisyear['week']<i)&(pbp_thisyear['week']>=i-5)]
      df_past4weeks=transform(df_temp)
    df_past4weeks=df_past4weeks.add_suffix('_4weeks')
    if i <= 3:
      df_temp=pbp_lastyear[pbp_lastyear['week']>9]
      df_past3weeks=transform(df_temp)
    else:
      df_temp=pbp_thisyear[(pbp_thisyear['week']<i)&(pbp_thisyear['week']>=i-3)]
      df_past3weeks=transform(df_temp)
    df_past3weeks=df_past3weeks.add_suffix('_3weeks')
    if i <= 2:
      df_temp=pbp_lastyear[pbp_lastyear['week']>9]
      df_past2weeks=transform(df_temp)
    else:
      df_temp=pbp_thisyear[(pbp_thisyear['week']<i)&(pbp_thisyear['week']>=i-2)]
      df_past2weeks=transform(df_temp)
    df_past2weeks=df_past2weeks.add_suffix('_2weeks')

    df_thisseason.fillna(0,inplace=True)
    fullteamdata_joined = fullteamdata.join(df_thisseason.join(df_past4weeks.join(df_past3weeks.join(df_past2weeks))))
    thisyear_temp = fullteamdata_joined.iloc[:,221:252]
    thisyear_temp.columns = thisyear_temp.columns.str.replace('_thisyear','')
    postthxgiving_temp = fullteamdata_joined.iloc[:,176:207]
    postthxgiving_temp.columns = postthxgiving_temp.columns.str.replace('postthxgiving_','')
    subtract = thisyear_temp.subtract(postthxgiving_temp).add_suffix('_difference')
    fullteamdata_joined = fullteamdata_joined.join(subtract)
    #make the time list
    # time_week = []
    # for a in range(0,378):
    #   time_week.append(i)
    # time_week

    for j in win_totals_temp[win_totals_temp['week']==i].index:
      away = fullteamdata_joined.loc[win_totals_temp.loc[j, 'away_team']]
      home = fullteamdata_joined.loc[win_totals_temp.loc[j, 'home_team']]
      df_away = pd.DataFrame(fullteamdata_joined.loc[win_totals_temp.loc[j, 'away_team']].add_suffix('_away')).rename(columns={win_totals_temp.loc[j, 'away_team']:'data'})
      df_home = pd.DataFrame(fullteamdata_joined.loc[win_totals_temp.loc[j, 'home_team']].add_suffix('_home')).rename(columns={win_totals_temp.loc[j, 'home_team']:'data'})
      df_difference = pd.DataFrame(away.subtract(home).add_suffix('_difference')).rename(columns={0:'data'})
      df = pd.concat([df_away,pd.concat([df_home,df_difference])])
      df = pd.concat([df, pd.DataFrame([i],index=['week'],columns=['data'])])
      df.fillna(0,inplace=True)
      count += 1

      if len(df.dropna(axis=0).index) < 1021:
        #column of error
        count_arr.append(count)



      df_list = list(df['data'])





      results.append([win_totals_temp.loc[j,'away_score'],win_totals_temp.loc[j,'home_score']])
      game_list_prev.append(df_list)
  year+=1
  fullteamdata.reset_index(inplace=True)

fullteamdata_arr.append(this_season)
# team = 'ARI'
# fullteamdata_2021_reset[fullteamdata_2021_reset['team']==team].reset_index()
# fullteamdata_2021.loc[team]
# week1_df =
len(game_list_prev)
# fullteamdata_2021
# game_list_2022
# game_list_2022

Streaming output truncated to the last 5000 lines.
<ipython-input-9-b3f885e8e658>:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['third_down_def'] = (third_down_transform.groupby('defteam').mean()[['third_down_converted']])
<ipython-input-9-b3f885e8e658>:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['ydsperplay'] = (pbp_rp_transform.groupby('posteam').mean()['yards_gained'])
<ipython-input-9-b3f885e8e658>:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only

1123

In [13]:
### create a list of dataframes for each game(2024)
game_list_2024 = []
training_data = []
results_2024 = []
# week1_df = pd.DataFrame(index=this_year.columns)
this_season.set_index('team',inplace=True)
pbp_thisyear=pbp_data[pbp_data['season']==2024]
pbp_lastyear=pbp_data[pbp_data['season']==2023]
win_totals_temp = win_totals[win_totals['season']==2024]
# win_totals_2021[win_totals_2021['week']==1].index
# count_bye = 0
# count_byetwoweeks = 0
thanksgiving = this_season.iloc[:,176:207]
thanksgiving.columns = thanksgiving.columns.str.replace('postthxgiving_','')
thanksgiving = thanksgiving.add_suffix('_thisyear')

###change this value!!!
current_week = 19
####

for i in range(1, current_week+1, 1):
  if i <= 4:
    df_thisseason = thanksgiving
    df_temp=pbp_lastyear[pbp_lastyear['week']>7]
    df_past5weeks=transform(df_temp)
  else:
    df_thisseason = thisseason(pbp_thisyear[pbp_thisyear['week']<i], win_totals_temp[win_totals_temp['week']<i])
    df_temp=pbp_thisyear[(pbp_thisyear['week']<i)&(pbp_thisyear['week']>=i-5)]
    df_past5weeks=transform(df_temp)
  df_past5weeks=df_past5weeks.add_suffix('_5weeks')
  if i <= 3:
    df_temp=pbp_lastyear[pbp_lastyear['week']>9]
    df_past3weeks=transform(df_temp)
  else:
    df_temp=pbp_thisyear[(pbp_thisyear['week']<i)&(pbp_thisyear['week']>=i-3)]
    df_past3weeks=transform(df_temp)
  df_past3weeks=df_past3weeks.add_suffix('_3weeks')
  if i <= 2:
    df_temp=pbp_lastyear[pbp_lastyear['week']>9]
    df_past2weeks=transform(df_temp)
  else:
    df_temp=pbp_thisyear[(pbp_thisyear['week']<i)&(pbp_thisyear['week']>=i-2)]
    df_past2weeks=transform(df_temp)
  df_past2weeks=df_past2weeks.add_suffix('_2weeks')
  # if i == 5:
  #   print(df_thisseason.iloc[:,30:40])
  df_thisseason.fillna(0)
  fullteamdata_joined = this_season.join(df_thisseason.join(df_past5weeks.join(df_past3weeks.join(df_past2weeks))))
  thisyear_temp = fullteamdata_joined.iloc[:,221:252]
  thisyear_temp.columns = thisyear_temp.columns.str.replace('_thisyear','')
  postthxgiving_temp = fullteamdata_joined.iloc[:,176:207]
  postthxgiving_temp.columns = postthxgiving_temp.columns.str.replace('postthxgiving_','')
  subtract = thisyear_temp.subtract(postthxgiving_temp).add_suffix('_difference')
  fullteamdata_joined = fullteamdata_joined.join(subtract)
  #make the time list

  for j in win_totals_temp[win_totals_temp['week']==i].index:
    away = fullteamdata_joined.loc[win_totals_temp.loc[j, 'away_team']]
    home = fullteamdata_joined.loc[win_totals_temp.loc[j, 'home_team']]
    df_away = pd.DataFrame(fullteamdata_joined.loc[win_totals_temp.loc[j, 'away_team']].add_suffix('_away')).rename(columns={win_totals_temp.loc[j, 'away_team']:'data'})
    df_home = pd.DataFrame(fullteamdata_joined.loc[win_totals_temp.loc[j, 'home_team']].add_suffix('_home')).rename(columns={win_totals_temp.loc[j, 'home_team']:'data'})
    df_difference = pd.DataFrame(away.subtract(home).add_suffix('_difference')).rename(columns={0:'data'})
    df = pd.concat([df_away,pd.concat([df_home,df_difference])])
    df = pd.concat([df, pd.DataFrame([i],index=['week'],columns=['data'])])
    df_list = list(df['data'])


    if i == current_week:
      game_list_2024.append(df_list)
    else:
      training_data.append(df_list)
      results_2024.append([win_totals_temp.loc[j,'away_score'],win_totals_temp.loc[j,'home_score']])

this_season.reset_index(inplace=True)


# team = 'ARI'
# fullteamdata_2021_reset[fullteamdata_2021_reset['team']==team].reset_index()
# fullteamdata_2021.loc[team]
# week1_df =

len(game_list_2024)
# fullteamdata_2021
# game_list_2022
# game_list_2022

<ipython-input-9-b3f885e8e658>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df_clean.groupby('posteam').mean()[['epa','wpa',]].rename(columns={'epa':'off_epa','wpa':'off_wpa'})
<ipython-input-9-b3f885e8e658>:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df[['def_epa','def_wpa']] = df_clean.groupby('defteam').mean()[['epa','wpa',]]
<ipython-input-9-b3f885e8e658>:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function

6

In [14]:
df_thisseason.iloc[:,8:]

,epa_off_thisyear,wpa_off_thisyear,epa_def_thisyear,wpa_def_thisyear,epa_off_4q_thisyear,wpa_off_4q_thisyear,epa_def_4q_thisyear,wpa_def_4q_thisyear,2ndydstogo_off_thisyear,2ndydstogo_def_thisyear,...,3rddownconverted_def_thisyear,points_pergame_off_thisyear,points_pergame_def_thisyear,yards_pergame_off_thisyear,yards_pergame_def_thisyear,win_percentage_thisyear,off_pointdifferential_thisyear,def_pointdifferential_thisyear,off_comp_percentage_thisyear,def_comp_percentage_thisyear
posteam,,,,,,,,,,,,,,,,,,,,,
ARI,0.074580,0.001948,0.071577,0.002838,0.071553,0.002355,0.048267,0.000857,7.643478,7.443804,...,0.441748,23.529412,22.294118,359.235294,343.352941,0.470588,-1.023219,1.308186,0.610114,0.598706
ATL,0.018536,0.001329,0.073925,0.002940,-0.152584,-0.004641,0.015464,-0.001203,7.529577,7.926346,...,0.454976,22.882353,24.882353,371.411765,347.000000,0.470588,-1.662800,1.742317,0.610738,0.618683
BAL,0.182991,0.004176,-0.007542,0.000793,0.153778,0.003945,0.035024,0.002381,7.921965,7.723032,...,0.394737,30.470588,21.235294,424.411765,325.000000,0.705882,3.855118,6.476448,0.585635,0.556180
BUF,0.162631,0.003659,0.007991,0.000162,0.091822,0.001240,-0.000497,-0.001374,7.738235,7.859649,...,0.437811,30.882353,21.647059,360.352941,342.117647,0.764706,2.116883,5.546318,0.568221,0.620690
CAR,-0.029567,0.001379,0.150534,0.004006,-0.115205,-0.001039,0.137015,0.004371,7.557276,7.224275,...,0.500000,20.058824,31.411765,298.705882,406.352941,0.294118,-7.000000,-4.159794,0.542994,0.605042
CHI,-0.049304,-0.001083,0.021739,0.001608,-0.009629,-0.000504,-0.129300,-0.000834,7.910112,7.908814,...,0.375587,18.235294,21.764706,284.647059,354.941176,0.294118,-5.327613,-3.108130,0.511594,0.571177
CIN,0.091495,0.004163,0.058187,0.003521,0.001600,0.000106,0.096570,0.003311,7.005682,7.570652,...,0.420290,27.764706,25.529412,366.411765,349.235294,0.529412,0.667438,3.450650,0.636238,0.571429
CLE,-0.150758,-0.002820,0.013101,0.001071,-0.140878,-0.002923,-0.028967,-0.001550,8.176630,8.000000,...,0.368182,15.176471,25.588235,301.705882,343.647059,0.176471,-7.530764,-4.764280,0.512987,0.545769
DAL,-0.046303,0.000908,0.087355,0.002646,-0.125965,0.002222,0.028627,0.002580,7.915300,7.532164,...,0.369458,20.588235,27.529412,328.235294,356.882353,0.411765,-4.607946,-1.358893,0.579330,0.583612


In [15]:
df_past5weeks

,off_epa_5weeks,off_wpa_5weeks,def_epa_5weeks,def_wpa_5weeks,offrp_epa_5weeks,offrp_wpa_5weeks,defrp_epa_5weeks,defrp_wpa_5weeks,ydstogo_2nd_off_5weeks,ydstogo2nd_def_5weeks,ydstogo_3rd_off_5weeks,ydstogo_3rd_def_5weeks,third_down_off_5weeks,third_down_def_5weeks,ydsperplay_5weeks,ydsperattempt_5weeks,ydsperrush_5weeks,opp_ydsperplay_5weeks,opp_ydsperattempt_5weeks,opp_ydsperrush_5weeks
posteam,,,,,,,,,,,,,,,,,,,,
ARI,0.092931,0.001332,0.069090,0.003397,0.100210,-0.000299,0.088746,0.003287,7.542857,7.752577,5.843750,7.306452,0.468750,0.387097,5.889908,6.043062,5.618644,6.067114,6.864706,5.007812
ATL,0.045340,0.002400,0.063761,0.004846,0.081168,0.003809,0.086314,0.006433,7.682243,8.074074,6.328125,8.016666,0.468750,0.466667,5.990854,7.048193,4.907407,5.634615,6.197115,4.509615
BAL,0.252666,0.004635,-0.237711,-0.003728,0.268837,0.004195,-0.313714,-0.004588,7.566265,7.763889,6.116279,6.260000,0.558140,0.280000,7.114754,8.765766,5.736842,4.398230,4.875000,3.418919
BUF,0.186103,0.003160,0.087527,0.001576,0.210316,0.004213,0.083678,0.001140,7.471154,7.381818,6.913793,7.983333,0.482759,0.533333,6.371710,7.948276,4.261539,5.879518,7.328432,3.570312
CAR,0.070679,0.005260,0.233632,0.006403,0.076300,0.005686,0.259091,0.005160,7.402062,7.500000,6.967742,5.830769,0.403226,0.584615,5.345891,5.875676,4.429906,6.482143,7.046243,5.883436
CHI,-0.088338,-0.003067,0.088756,-0.000018,-0.146151,-0.005061,0.105173,-0.000070,7.910000,7.606061,7.983606,6.322581,0.311475,0.451613,4.262976,4.775401,3.323529,6.159609,7.606250,4.585034
CIN,0.118580,0.007054,-0.044850,0.000299,0.114314,0.006714,-0.085042,-0.004389,6.500000,8.448598,5.901639,7.982143,0.491803,0.357143,5.829341,6.660793,4.065421,4.904605,5.151042,4.482143
CLE,-0.287310,-0.006138,0.028825,0.000956,-0.348544,-0.006862,-0.002241,0.000520,8.471154,7.723810,8.090909,6.632353,0.303030,0.382353,4.196262,3.883178,4.822430,5.265372,6.162921,4.045802
DAL,0.041553,0.002056,0.088157,0.003742,-0.025169,0.000616,0.067476,0.002871,8.085714,7.551021,6.385714,7.830509,0.400000,0.423729,5.396825,6.142857,4.544218,5.837838,6.463542,4.682693


In [16]:
win_totals_temp[win_totals_temp['week']==current_week-1]

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium,away_win,home_win
6962,2024_18_CLE_BAL,2024,REG,18,2025-01-04,Saturday,16:30,CLE,10.0,BAL,...,00-0034796,Bailey Zappe,Lamar Jackson,Kevin Stefanski,John Harbaugh,Scott Novak,BAL00,M&T Bank Stadium,0,1
6963,2024_18_CIN_PIT,2024,REG,18,2025-01-04,Saturday,20:00,CIN,19.0,PIT,...,00-0029263,Joe Burrow,Russell Wilson,Zac Taylor,Mike Tomlin,John Hussey,PIT00,Acrisure Stadium,1,0
6964,2024_18_CAR_ATL,2024,REG,18,2025-01-05,Sunday,13:00,CAR,44.0,ATL,...,00-0039917,Bryce Young,Michael Penix Jr.,Dave Canales,Raheem Morris,Land Clark,ATL97,Mercedes-Benz Stadium,1,0
6965,2024_18_WAS_DAL,2024,REG,18,2025-01-05,Sunday,13:00,WAS,23.0,DAL,...,00-0037012,Jayden Daniels,Trey Lance,Dan Quinn,Mike McCarthy,Clete Blakeman,DAL00,AT&T Stadium,1,0
6966,2024_18_CHI_GB,2024,REG,18,2025-01-05,Sunday,13:00,CHI,24.0,GB,...,00-0036264,Caleb Williams,Jordan Love,Matt Eberflus,Matt LaFleur,Clay Martin,GNB00,Lambeau Field,1,0
6967,2024_18_JAX_IND,2024,REG,18,2025-01-05,Sunday,13:00,JAX,23.0,IND,...,00-0026158,Mac Jones,Joe Flacco,Doug Pederson,Shane Steichen,Alex Kemp,IND00,Lucas Oil Stadium,0,1
6968,2024_18_BUF_NE,2024,REG,18,2025-01-05,Sunday,13:00,BUF,16.0,NE,...,00-0039398,Mitchell Trubisky,Joe Milton III,Sean McDermott,Jerod Mayo,Alan Eck,BOS00,Gillette Stadium,0,1
6969,2024_18_NYG_PHI,2024,REG,18,2025-01-05,Sunday,13:00,NYG,13.0,PHI,...,00-0038400,Drew Lock,Tanner McKee,Brian Daboll,Nick Sirianni,Ron Torbert,PHI00,Lincoln Financial Field,0,1
6970,2024_18_NO_TB,2024,REG,18,2025-01-05,Sunday,13:00,NO,19.0,TB,...,00-0034855,Spencer Rattler,Baker Mayfield,Dennis Allen,Todd Bowles,Adrian Hill,TAM00,Raymond James Stadium,0,1
6971,2024_18_HOU_TEN,2024,REG,18,2025-01-05,Sunday,13:00,HOU,23.0,TEN,...,00-0034771,Davis Mills,Mason Rudolph,DeMeco Ryans,Brian Callahan,Brad Allen,NAS00,Nissan Stadium,1,0


In [17]:
print(len(game_list_prev + training_data))

1395


In [35]:
## the model


sum_arr = np.array([])

#concatenate past year's data and this year's data
training_x = game_list_prev + training_data
training_y = results + results_2024

### create and fit the model
from sklearn.neural_network import MLPRegressor
from sklearn import model_selection
from sklearn.metrics import r2_score


for i in range(0,10):

  ### play around with hidden layer sizes and check validation results
  # if current_week < 5:
  #   prediction_model = MLPRegressor(hidden_layer_sizes=(200,110,60), max_iter=17)
  # else:
  #   prediction_model = MLPRegressor(hidden_layer_sizes=(195,110,60), max_iter=17)
  prediction_model = MLPRegressor(hidden_layer_sizes=(195,110,60), activation='relu',max_iter=17)
  prediction_model.fit(training_x,training_y)#185,110,60,  16
  ###validation
  pred = prediction_model.predict(game_list_prev)
  r2score = r2_score(results, pred)
  # prediction_model.predict([first_game])
  print(r2score)
  # pred
  if i > 0:
    sum_arr = np.array(prediction_model.predict(game_list_2024)) + sum_arr
  else:
    sum_arr = np.array(prediction_model.predict(game_list_2024))


arr = sum_arr/10

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (17) reached and the optimization hasn't converged yet.
  warnings.warn(


0.14074850606169353


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (17) reached and the optimization hasn't converged yet.
  warnings.warn(


0.18078101532341945


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (17) reached and the optimization hasn't converged yet.
  warnings.warn(


...

0.1961053511512062


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (17) reached and the optimization hasn't converged yet.
  warnings.warn(


0.14783314560941058


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (17) reached and the optimization hasn't converged yet.
  warnings.warn(


0.16632664210441905


In [36]:
### predictions in table format


df = pd.DataFrame(arr, columns=['away_score', 'home_score'])
df.insert(0, 'game', win_totals_temp[win_totals_temp['week']==current_week][['game_id']].set_index(df.index))
df

,game,away_score,home_score
0,2024_19_LAC_HOU,21.980393,23.228774
1,2024_19_PIT_BAL,19.599268,29.853179
2,2024_19_DEN_BUF,21.798327,27.025894
3,2024_19_GB_PHI,22.446645,22.631932
4,2024_19_WAS_TB,22.180255,24.638365
5,2024_19_MIN_LA,22.156263,21.691571


In [ ]:
#average point differential for the home team


# sum_test = 0
# for arr in training_y:
#   sum_test += arr[1] - arr[0]
# home_advantage = sum_test/len(training_y)
# home_advantage

In [ ]:
# only use for super bowl (neutral site):


# arr = prediction_model.predict(game_list_2024)
# arr[0][0] = arr[0][0] + home_advantage/2
# arr[0][1] = arr[0][1] - home_advantage/2
# df = pd.DataFrame(arr, columns=['away_score', 'home_score'])
# df.insert(0, 'game', win_totals_temp[win_totals_temp['week']==current_week][['game_id']].set_index(df.index))
# df

In [ ]:

###test
# depth_chart[(depth_chart['week'] == 1)&(depth_chart['club_code']=='TB')]
# depth_chart.iloc[:,20:30]
#multiple years?
# pbp_list = nfl.import_pbp_data([2015,2016,2017,2018,2019,2020,2021,2022])
# depth_chart[(depth_chart['position']=='QB')&(depth_chart['club_code']=='NE')&(depth_chart['week']==22)]
# pbp_list
#rookie case
# seasonal_data[seasonal_data['player_id']==00000]['completions']
# pbp_rp_2021[pbp_rp_2021['pass']==1].groupby('posteam')['epa'].mean()

# postthxgiving_temp['epa/db_diff']
